Le **maps** consentono di trasformare i dati in un DataFrame o in una Serie un valore alla volta per un'intera colonna. Tuttavia, spesso vogliamo raggruppare i nostri dati e poi fare qualcosa di specifico per il gruppo in cui si trovano.

Come si apprenderà, è possibile farlo con l'operazione `groupby()`. Verranno trattati anche altri argomenti, come i modi più complessi per indicizzare i DataFrames e come ordinare i dati.

Per iniziare l'esercizio di questo argomento, fate clic qui.

## Analisi Groupwise
Una funzione che abbiamo utilizzato molto finora è la funzione `value_counts()`. Possiamo replicare ciò che fa `value_counts()` facendo come segue:

In [1]:
import pandas as pd
reviews = pd.read_csv("D:/Users/Alessio/OneDrive/Python/Kaggle/Pandas/winemag-data-130k-v2.csv", index_col=0)
pd.set_option("display.max_rows", 5)

In [2]:
reviews.groupby('points').points.count()

points
80     397
81     692
      ... 
99      33
100     19
Name: points, Length: 21, dtype: int64

`groupby()` crea un gruppo di recensioni che assegnano lo stesso valore di punti ai vini indicati. Poi, per ognuno di questi gruppi, abbiamo preso la colonna `points()` e contato quante volte è apparsa. `value_counts()` è solo una scorciatoia per questa operazione `groupby()`.

Con questi dati possiamo utilizzare qualsiasi funzione di riepilogo che abbiamo usato in precedenza. Ad esempio, per ottenere il vino più economico in ogni categoria di valore, possiamo procedere come segue:

In [3]:
reviews.groupby('points').price.min()

points
80      5.0
81      5.0
       ... 
99     44.0
100    80.0
Name: price, Length: 21, dtype: float64

Si può pensare che ogni gruppo generato sia una fetta del nostro DataFrame contenente solo i dati con valori corrispondenti. Questo DataFrame è accessibile direttamente con il metodo `apply()` e possiamo manipolare i dati nel modo che riteniamo più opportuno. Per esempio, ecco un modo per selezionare il nome del primo vino recensito da ogni cantina del dataset:

In [4]:
reviews.groupby('winery').apply(lambda df: df.title.iloc[0])

winery
1+1=3                          1+1=3 NV Rosé Sparkling (Cava)
10 Knots                 10 Knots 2010 Viognier (Paso Robles)
                                  ...                        
àMaurice    àMaurice 2013 Fred Estate Syrah (Walla Walla V...
Štoka                         Štoka 2009 Izbrani Teran (Kras)
Length: 16757, dtype: object

Per un controllo ancora più preciso, si può anche raggruppare per più di una colonna. Per esempio, ecco come scegliere il vino migliore in base al Paese e alla provincia:

In [5]:
reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])

country  \
country   province                      
Argentina Mendoza Province  Argentina   
          Other             Argentina   
...                               ...   
Uruguay   San Jose            Uruguay   
          Uruguay             Uruguay   

                                                                  description  \
country   province                                                              
Argentina Mendoza Province  If the color doesn't tell the full story, the ...   
          Other             Take note, this could be the best wine Colomé ...   
...                                                                       ...   
Uruguay   San Jose          Baked, sweet, heavy aromas turn earthy with ti...   
          Uruguay           Cherry and berry aromas are ripe, healthy and ...   

                                          designation  points  price  \
country   province                                                     
Argentina Mendoza Province           Nicasia Vineyard      97  120.0   
          Other                               Reserva      95   90.0   
...                                               ...     ...    ...   
Uruguay   San Jose           El Preciado Gran Reserva      87   50.0   
          Uruguay           Blend 002 Limited Edition      91   22.0   

                                    province region_1 region_2  \
country   province                                               
Argentina Mendoza Province  Mendoza Province  Mendoza      NaN   
          Other                        Other    Salta      NaN   
...                                      ...      ...      ...   
Uruguay   San Jose                  San Jose      NaN      NaN   
          Uruguay                    Uruguay      NaN      NaN   

                                  taster_name taster_twitter_handle  \
country   province                                                    
Argentina Mendoza Province  Michael Schachner           @wineschach   
          Other             Michael Schachner           @wineschach   
...                                       ...                   ...   
Uruguay   San Jose          Michael Schachner           @wineschach   
          Uruguay           Michael Schachner           @wineschach   

                                                                        title  \
country   province                                                              
Argentina Mendoza Province  Bodega Catena Zapata 2006 Nicasia Vineyard Mal...   
          Other                            Colomé 2010 Reserva Malbec (Salta)   
...                                                                       ...   
Uruguay   San Jose          Castillo Viejo 2005 El Preciado Gran Reserva R...   
          Uruguay           Narbona NV Blend 002 Limited Edition Tannat-Ca...   

                                          variety                winery  
country   province                                                       
Argentina Mendoza Province                 Malbec  Bodega Catena Zapata  
          Other                            Malbec                Colomé  
...                                           ...                   ...  
Uruguay   San Jose                      Red Blend        Castillo Viejo  
          Uruguay           Tannat-Cabernet Franc               Narbona  

[425 rows x 13 columns]

Un altro metodo `groupby()` degno di nota è `agg()`, che consente di eseguire contemporaneamente una serie di funzioni diverse sul DataFrame. Ad esempio, possiamo generare un semplice riepilogo statistico dell'insieme di dati come segue:

In [6]:
reviews.groupby(['country']).price.agg([len, min, max])

,len,min,max
country,,,
Argentina,3800,4.0,230.0
Armenia,2,14.0,15.0
...,...,...,...
Ukraine,14,6.0,13.0
Uruguay,109,10.0,130.0


Un uso efficace di `groupby()` permette di fare molte cose potenti con il vostro set di dati.

## Indici multipli
In tutti gli esempi visti finora, abbiamo lavorato con oggetti DataFrame o Series con un indice a singola etichetta. `groupby()` è leggermente diverso, in quanto, a seconda dell'operazione che eseguiamo, a volte produce un cosiddetto **indice multiplo**.

Un **indice multiplo** si differenzia da un indice normale perché ha più livelli. Ad esempio:

In [7]:
countries_reviewed = reviews.groupby(['country', 'province']).description.agg([len])
countries_reviewed

len
country   province              
Argentina Mendoza Province  3264
          Other              536
...                          ...
Uruguay   San Jose             3
          Uruguay             24

[425 rows x 1 columns]

In [8]:
mi = countries_reviewed.index
type(mi)

pandas.core.indexes.multi.MultiIndex

Gli **indici multipli** dispongono di diversi metodi per gestire la loro struttura a livelli, assenti negli indici a livello singolo. Inoltre, richiedono due livelli di etichette per recuperare un valore. La **gestione dell'output di un indice multiplo** è un problema comune per gli utenti che non conoscono pandas.

I casi d'uso di un indice multiplo sono descritti in dettaglio, insieme alle istruzioni per il loro utilizzo, nella sezione **MultiIndex / Advanced Selection** della documentazione di pandas.

Tuttavia, in generale, il **metodo multi-indice** che si utilizzerà più spesso è quello per la conversione in un indice regolare, il metodo `reset_index()`:

In [9]:
countries_reviewed.reset_index()

,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
...,...,...,...
423,Uruguay,San Jose,3
424,Uruguay,Uruguay,24


## Sorting
Osservando nuovamente `countries_reviewed` si può notare che il raggruppamento restituisce i dati in ordine di indice, non in ordine di valore. In altre parole, quando si produce il risultato di un `groupby`, l'ordine delle righe dipende dai valori dell'indice, non dai dati.

Per ottenere i dati nell'ordine desiderato, possiamo **ordinarli** noi stessi. Il `metodo sort_values()` è utile a questo scopo.

In [10]:
countries_reviewed = countries_reviewed.reset_index()
countries_reviewed.sort_values(by='len')

,country,province,len
179,Greece,Muscat of Kefallonian,1
192,Greece,Sterea Ellada,1
...,...,...,...
415,US,Washington,8639
392,US,California,36247


`sort_values()` ha come impostazione predefinita un **ordinamento ascendente**, in cui i valori più bassi vanno per primi. Tuttavia, la maggior parte delle volte vogliamo un **ordinamento discendente**, in cui i numeri più alti vanno per primi, `ascending=False`. Così si fa:

In [11]:
countries_reviewed.sort_values(by='len', ascending=False)

,country,province,len
392,US,California,36247
415,US,Washington,8639
...,...,...,...
63,Chile,Coelemu,1
149,Greece,Beotia,1


Per ordinare in base ai **valori dell'indice**, utilizzare il metodo companion `sort_index()`. Questo metodo ha gli stessi argomenti e lo stesso ordine predefinito:

In [12]:
countries_reviewed.sort_index()

,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
...,...,...,...
423,Uruguay,San Jose,3
424,Uruguay,Uruguay,24


Infine, è possibile ordinare per più colonne alla volta:

In [13]:
countries_reviewed.sort_values(by=['country', 'len'])

,country,province,len
1,Argentina,Other,536
0,Argentina,Mendoza Province,3264
...,...,...,...
424,Uruguay,Uruguay,24
419,Uruguay,Canelones,43


## Esercizi
In questi esercizi applicheremo l'**analisi groupwise** al nostro set di dati.

Eseguire la cella di codice sottostante per caricare i dati prima di eseguire gli esercizi.

In [16]:
import pandas as pd

reviews = pd.read_csv("D:/Users/Alessio/OneDrive/Python/Kaggle/Pandas/winemag-data-130k-v2.csv", index_col=0)
pd.set_option("display.max_rows", 5)

### Domanda 1
Chi sono i recensori di vino più comuni nel dataset? Creare una `Serie` il cui indice è la categoria `taster_twitter_handle` del dataset e i cui valori contano quante recensioni ha scritto ogni persona.

In [25]:
# Your code here
reviews_written = reviews.groupby('taster_twitter_handle').size()
        # entrambi giusti
reviews_written = reviews.groupby('taster_twitter_handle').taster_twitter_handle.count()

# Check your answer
# q1.check()
reviews_written

taster_twitter_handle
@AnneInVino        3685
@JoeCz             5147
                   ... 
@winewchristina       6
@worldwineguys     1005
Name: taster_twitter_handle, Length: 15, dtype: int64

### Domanda 2
Qual è il miglior vino che posso comprare per una data somma di denaro? Creare una `Serie` il cui indice è il prezzo del vino e il cui valore è il numero massimo di punti assegnati a un vino di quel prezzo in una recensione. Ordinare i valori per prezzo, in ordine crescente (in modo che `4,0` dollari sia in cima e `3300,0` dollari in fondo).

In [26]:
best_rating_per_price = reviews.groupby('price')['points'].max().sort_index()

# Check your answer
# q2.check()
best_rating_per_price

price
4.0       86
5.0       87
          ..
2500.0    96
3300.0    88
Name: points, Length: 390, dtype: int64

### Domanda 3
Quali sono i prezzi minimi e massimi per ogni `variety` di vino? Creare un `DataFrame` il cui indice è la categoria `variety` del dataset e i cui valori sono i valori `min` e `max` della stessa.

In [28]:
price_extremes = reviews.groupby('variety').price.agg([min, max])

# Check your answer
# q3.check()
price_extremes

,min,max
variety,,
Abouriou,15.0,75.0
Agiorgitiko,10.0,66.0
...,...,...
Çalkarası,19.0,19.0
Žilavka,15.0,15.0


### Domanda 4
Quali sono le varietà di vino più costose? Creare una variabile `sorted_variety` contenente una copia del dataframe della domanda precedente, in cui le varietà sono ordinate in ordine decrescente in base al prezzo minimo e poi al prezzo massimo (per rompere i legami).

In [29]:
sorted_varieties = price_extremes.sort_values(by=['min', 'max'], ascending=False)

# Check your answer
# q4.check()
sorted_varieties

,min,max
variety,,
Ramisco,495.0,495.0
Terrantez,236.0,236.0
...,...,...
Vital,NaN,NaN
Zelen,NaN,NaN


### Domanda 5
Creare una `Serie` il cui indice è recensori e il cui valore è la media dei punteggi delle recensioni rilasciate da quel recensore. Suggerimento: sono necessarie le colonne `taster_name` e `points`.

In [30]:
reviewer_mean_ratings = reviews.groupby('taster_name').points.mean()

# Check your answer
# q5.check()
reviewer_mean_ratings

taster_name
Alexander Peartree    85.855422
Anna Lee C. Iijima    88.415629
                        ...    
Susan Kostrzewa       86.609217
Virginie Boone        89.213379
Name: points, Length: 19, dtype: float64

Ci sono differenze significative nei punteggi medi assegnati dai vari recensori? Eseguite la cella sottostante per utilizzare il metodo describe() e vedere un riepilogo della gamma di valori.

### Domanda 6
Quale combinazione di paesi e varietà è più comune? Creare una `Serie` il cui indice è un `MultiIndex` di coppie `{country, variety}`. Ad esempio, un pinot nero prodotto negli Stati Uniti dovrebbe corrispondere a `{"US", "Pinot Noir"}`. Ordinare i valori nella `Serie` in ordine decrescente in base al numero di vini.

In [31]:
country_variety_counts = reviews.groupby(['country', 'variety']).size().sort_values(ascending=False)

# Check your answer
# q6.check()
country_variety_counts

country  variety           
US       Pinot Noir            9885
         Cabernet Sauvignon    7315
                               ... 
Mexico   Rosado                   1
Uruguay  White Blend              1
Length: 1612, dtype: int64